##### Training DeepCDR with the new data generator

So far we have written down a new generator function with tf.keras.Sequence, and we have trained the DeepCDR model with this function. We however still need to write an improve-compliant code if we are to share the work with people from ANL. Therefore, let's try doing that with the new generator function for both the train and inference scripts.

The generator function is available in the py script "New_data_generator_with_tf.py", and the train script with this function is available at "DeepCDR_train_with_new_generator.py". 

Okay, we have made the two scripts improve compliant: The codes are stored as; train script - "deepcdr_train_improve_with_new_generator.py" and infer script - "deepcdr_infer_improve_with_new_data_generator.py".

One more thing to do here before we get to the bootstrapping method and storing the predictions. See if the train and validation steps are necessary, or if the written generator will automatically complete training at the end of the required number of batches in each epoch

#### Bootstrap CIs for DeepCDR

##### Training bootsrap samples for DeepCDR for UQ

Okay, we can now move to bootstrap sample training. We should adjust our code in a way that the trained model and the validation scores from the exercise can be stored, so that later we can use these models to compute the CIs and talk about uncertainity.

Let's use the bootstrap generator, and compare it against the original generator, to make sure the bootstrap sampling generator is in fact doing what it is supposed to do. This comparison is done in the 'Examine_bootstrap_samples.ipynb'.

We ensured that bootstrap samples indeed give different results than the regular training batches, so I think we are finally ready to implement the bootstrap training for DeepCDR for the UQ project - To get the predictions for the rrain data for the different botstrap samples, and then train the NNe, and then get the CIs, and thereby talk about the width and the coverages of the confidence intervals.

Notice that, we add the bootstrap data generator class to the python script "New_data_generator_with_tf.py" so that it would seem less complex to someone going through the codes (else too many python scripts, eventhough they are just simple fucntion r class definitions).

The bootstrap train code is in the python script "DeepCDR_train_with_new_generator_bootstraps.py". This is a generic code, and is still not the improve-compliant one. This code defines the validation data generator (that is created by the regular generator construct and not the bootstrap), and the folder to store the bootstrap results outside the for loop for training the bootstrap models. Inside the for loop, it first defines the model, compile the model, train the model, save the model, evaluate the model, and also stores the validation predictions and the associated scores. However, we cannot store the predictions or the scores for the train data for the bootstrap work due to an assertion error that we get in the improve library (it basically checks if the train order of the predicted values is the same, but since in bootstraps we have some samples repeated, we will not be able to use the improve methods to store the predictions for the train data. Therefore, we will do this step in the next model train - NNe work).

Our next step will be to write improve-compliant code for training the bootrap models. 

The improve-compliant bootstrap training of the DeepCDR model is in the py script "deepcdr_train_bootstrap_improve_with_new_generator.py". Note that the build model path [Req] in the original code is now defined inside the for loop. We still need to make sure that this code works, and then we should be good to get started with developing the code for the model NNe.

##### Training NNE

For this, we need the predictions from the trained model, and the respective quantities computed. The model we can use for the training will have to be the same model we used for DeepCDR model, as we have several sets of inputs for the exercise.

We will initially develop some work in the ipynb "Train_NNe_and_PIs.ipynb", and later put all the codes together, and also write an improve-compliant code later.

"Train_NNe_and_PIs.py" has the end to end code for training the NNe, and the trained NNe model is saved in the folder "NNe_model".

We still need to make this train script improve-compliant, if we are to sahre with them, and mark what all changes to be made - for both main train scripts of the models, and train script of the NNe model.

##### Getting the PIs for the test data

We have done this in the same python scripts as above, the Train_NNe_and_PIs.py script and also the rough work in the corresponding notebook. And I think we can use this script as the infer script for improve, we just have to make it improve-compliant now.

The improve compliant code is written in "deepcdr_infer_bootstrap_imporve_with_new_generator.py" script. Note that this also stores the coverage and width in a json file in the output directory.

Okay, so training and infer is there, what abobut the preprocessing part? I think we never made changes in there, so wew use the same old script?

##### Steps for running the py scripts on the command line for getting the bootstrap predictions and the corresponding predicton intervals aong with their coverages and widths.

1. For the preprocessing script: python deepcdr_preprocess_improve.py --input_dir ./csa_data/raw_data --output_dir exp_result

This will store all preprocessed data in the location(folder) exp_result. The contents here at this point will be:

exp_result

 ├── param_log_file.txt
 
 ├── cancer_dna_methy_model
 
 ├── cancer_gen_expr_model
 
 ├── cancer_gen_mut_model
 
 ├── test_y_data.csv
 
 ├── train_y_data.csv
 
 ├── val_y_data.csv
 
 ├── drug_features.pickle
 
 └── norm_adj_mat.pickle

2. For the train script: python deepcdr_train_bootstrap_improve_with_new_generator.py --input_dir exp_result

The contents of this folder will look like below.

bootstrap_results_all

 ├── bootstrap_1
 
      └── DeepCDR_model

      └── val_scores.json
    
      └── val_y_data_predicted.csv
 
 ├── bootstrap_2
 
      └── DeepCDR_model

      └── val_scores.json

      └── val_y_data_predicted.csv
 
.

.

.
 
 ├── bootstrap_10
 
      └── DeepCDR_model

      └── val_scores.json

      └── val_y_data_predicted.csv



3. python deepcdr_infer_bootstrap_improve_with_new_generator.py --input_data_dir exp_result --input_model_dir bootstrap_results_all --output_dir bootstrap_inference --calc_infer_score true

The contents from this step will be as follows

bootstrap_results_all

    └── param_log_file.txt

    └── test_scores.json
    
    └── CI_information_bootstraps.json

    └── test_y_data_predicted.csv

NNe_model


We also need a submit file for execution of the train and infer functions of very large datasets.

For the large datasets, it takes so much time to train and also it requires a lot of memory, so we must submit them with a higher RAM as a job.

#### Dropout induced CIs for DeepCDR

For this work, we do nobt have to change the preprocess and the train script. We only need to adjust the inference script - should not be too bad. However, we still want to do the generator in the train script with the new generator we defined with the tf.keras.utils.Sequence class stored in the py script "New_data_generator_with_tf" (DataGenerator class for this to be exact).

Let's just try it for the CCLE dataset maybe -  we know that this method is not the best to do the CI analysis, so we should be okay just doing it for one. 

We do have the preprocessed data for this CCLE split 0 in the directories about CCLE, but let's store this work separately? - So that we have clarity when we get back here. Let's call this folder mc_dropout_exp_results?

So no changes to the preprocess and the train scripts. We do have to adjust the test script to make it improve compliant. Infer_with_uq_exp_CCLE.ipynb notebook has the complete inference code, let's first take it on to a python script and later make it improve-compliant. The corresponding python script is Infer_with_uq_exp_CCLE.py in the folder. We can now work on making it improve-compliant.

The improve compliant inference script is named "deepcdr_infer_mcdropout_improve.py". Note that this file also stores the width and the coverage information in a json file that get stored into the mc_dropout_exp_results folder. The lines below can be used to run the work on a terminal.

Running code on HCC - go to a terminal, go inside the location where the scripts are stored.

on terminal:

module load anaconda

conda activate $COMMON/deepcdr_improve_env

source setup_improve.sh


1. Run the preprocess script - no changes to the script

python deepcdr_preprocess_improve.py --input_dir ./csa_data/raw_data --output_dir mc_dropout_exp_results

2. Run the train script - just change where teh generator functions are coming from - old train script with yield generator should work fine as well, but let's for now go with the new generator function - this is the same exact script as deepcdr_train_improve_with_new_generator.py. We named it deepcdr_train_mcdropout_improve.py

python deepcdr_train_mcdropout_improve.py --input_dir mc_dropout_exp_results --output_dir mc_dropout_exp_results

Running the above two scripts should be a no-brainer as there aren't any adjustments to be done

3. What about the inference script?


python deepcdr_infer_mcdropout_improve.py --input_data_dir mc_dropout_exp_results --input_model_dir mc_dropout_exp_results --output_dir mc_dropout_exp_results --calc_infer_score true

The files that get saved in these steps are smilar to the ones we get in our original work. We also get an additional json file that stores the CI information (coverage and width) in the output directory.